In [2]:
!pip install transformers
!pip install pandas
!pip install tesseract-ocr
!pip install "unstructured[pdf]"
!pip install -U sentence-transformers
!pip install jupyter-black
!pip install plotly
!pip install scikit-learn



  Using cached tesseract-ocr-0.0.1.tar.gz (33 kB)
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Obtaining dependency information for cython from https://files.pythonhosted.org/packages/cf/fe/5c927644e1bb14c73714393fce3a9f96076f325c7182e9791f9acfc61a9c/Cython-3.0.3-cp310-cp310-macosx_10_9_x86_64.whl.metadata
  Using cached Cython-3.0.3-cp310-cp310-macosx_10_9_x86_64.whl.metadata (3.2 kB)
Using cached Cython-3.0.3-cp310-cp310-macosx_10_9_x86_64.whl (3.1 MB)
  error: subprocess-exited-with-error
  
  × Building wheel for tesseract-ocr (pyproject.toml) did not run successfully.
  │ exit code: 1
  ╰─> [30 lines of output]
      /private/var/folders/n3/p35jmksd0jj9c0lhvz2s6r480000gp/T/pip-build-env-rlprwxvx/overlay/lib/python3.10/site-packages/setuptools/dist.py:498: SetuptoolsDeprecationWarning: Invalid dash-separated options
      !!
      
              **********************************************

In [3]:
%load_ext jupyter_black


In [4]:
from unstructured.cleaners.core import replace_unicode_quotes
from unstructured.partition.pdf import partition_pdf
import re
from unstructured.staging.base import convert_to_dataframe

In [5]:
# Returns a List[Element] present in the pages of the parsed pdf document
elements = partition_pdf("data/claims_example.pdf")

In [7]:
df = convert_to_dataframe(elements)
df.head(5)

,type,text,element_id,filename,file_directory,last_modified,filetype,page_number,coordinates_points,coordinates_system,coordinates_layout_width,coordinates_layout_height
0,Title,HISTORY,23e04084df125efcc954cba154e2f4c9,claims_example.pdf,data,2023-07-07T10:09:51,application/pdf,1,"((369.6231838215999, 79.65664610791805), (369....",PixelSpace,595.29,841.89
1,Title,STATEMENT OF NO CLAIMS BONUS,02277faf1fefcb17410fdec82d1d82e4,claims_example.pdf,data,2023-07-07T10:09:51,application/pdf,1,"((175.17355045399998, 80.18085960000008), (175...",PixelSpace,595.29,841.89
2,Title,Policy Number MR321597307,c2a98b6fc974f4b510da1f266a3c4d16,claims_example.pdf,data,2023-07-07T10:09:51,application/pdf,1,"((37.411897333999995, 118.58132039999998), (37...",PixelSpace,595.29,841.89
3,Title,Renew al Date 11-12-2016,ee6a3419331716673f40f44c02d2628e,claims_example.pdf,data,2023-07-07T10:09:51,application/pdf,1,"((209.97396805399998, 118.58132039999998), (20...",PixelSpace,595.29,841.89
4,Title,Date Of Issue 16-11-2016,2a7d85a8aa7df1cac50336f312e1b77f,claims_example.pdf,data,2023-07-07T10:09:51,application/pdf,1,"((394.53618277399994, 118.58132039999998), (39...",PixelSpace,595.29,841.89


In [9]:
from unstructured.cleaners.extract import extract_datetimetz

remove_citations = lambda text: re.sub("ddddd", "", text)

test_el = elements[5]
test_el.apply(remove_citations)

# elements_sub = [element(remove_citations) for element in elements]
# print(test_el)

el_text = [el.text for el in elements]
doc_text = " ".join(el_text)

print(doc_text)

# how many words
len(doc_text.split(" "))

# duplicate for exmaple
doc_ds = [doc_text, doc_text]


# idea for later: get mulitple documents of same type with different policy holder to extract NER text

HISTORY STATEMENT OF NO CLAIMS BONUS Policy Number MR321597307 Renew al Date 11-12-2016 Date Of Issue 16-11-2016 Policyholder Details Eimear Nolan Ballinastraw Agent/Broker Details McCarthy Insurance Group Anderson Centre O' Niel Crowley Quay Fermoy, Co. Cork Ireland Bunclody Co Wexford No Claim Discount Scale applicable to your policy:- Entry 0% Stage 1 10% Stage 2 20% Stage 3 30% Stage 4 40% Stage 5 50%50% TO WHOM IT MAY CONCERN, This is to confirm that the above named insured is entitled to a 50% No Claims Discount on Vehicle registration/identification number 02WX1202 as at 16/11/2016 representing Stage 1* on our No Claim Discount Scale. The actual number of years claims-free on this policy are 1.** Note: The details above are based on the information known to us at the date of issue. BRIAN MAHON Chief Underwriting Officer Aviva Insurance Limited ** Please note that the number of years claim-free indicated above may not match the stage on our No Claim Discount Scale (marked *) abov

In [10]:
from transformers import BertTokenizer

# Load the tokenizer
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

# Tokenize the doc
tokens = tokenizer.tokenize(doc_text)

# Get the number of tokens
num_tokens = len(tokens)

print(f"Number of tokens in insurance doc : {num_tokens}")

Number of tokens in insurance doc : 467


In [11]:
from transformers import AutoTokenizer, AutoModel
import torch

# Load model from HuggingFace Hub
tokenizer = AutoTokenizer.from_pretrained("BAAI/bge-base-en-v1.5")
model = AutoModel.from_pretrained("BAAI/bge-base-en-v1.5")
model.eval()

# max_tokens = 512 <- will need to change this to a bigger model and clean unrequired text


# Tokenize sentences
encoded_input = tokenizer(doc_ds, padding=True, truncation=True, return_tensors="pt")

# Compute token embeddings
with torch.no_grad():
    model_output = model(**encoded_input)
    # Perform pooling. In this case, cls pooling.
    sentence_embeddings = model_output[0][:, 0]
# normalize embeddings
sentence_embeddings = torch.nn.functional.normalize(sentence_embeddings, p=2, dim=1)
print("Sentence embeddings:", sentence_embeddings)

Sentence embeddings: tensor([[-0.0265, -0.0122,  0.0430,  ...,  0.0173, -0.0176, -0.0083],
        [-0.0265, -0.0122,  0.0430,  ...,  0.0173, -0.0176, -0.0083]])


In [94]:
import plotly.express as px
from sklearn.manifold import TSNE

# Reduce dimensionality using t-SNE
tsne = TSNE(n_components=2, random_state=42, n_iter=1000, metric="cosine", perplexity=1)

tsne_results = tsne.fit_transform(sentence_embeddings)

# Visualize using Plotly
df = {"x": tsne_results[:, 0], "y": tsne_results[:, 1], "sentence": sentence_embeddings}
fig = px.scatter(df, x="x", y="y", hover_data=["sentence"])
fig.show()

: 

: 